# torchattacks demo

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torchvision.utils import save_image
import torchattacks
import os


## Preprocessing

### Load pretraiend model resnet56 from torch.hub

In [2]:

# Here is the selection: Loading from Github or local source.

#model = torch.hub.load("pytorch/vision:v0.13.0", "vgg16_bn", pretrained=True)
model=torchvision.models.vgg19(pretrained=True)

c:\Users\19401\anaconda3\envs\PRP\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
c:\Users\19401\anaconda3\envs\PRP\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Load dataset from imagenet

In [3]:

use_cuda = True

norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]
transform = transforms.Compose(
    [transforms.Resize((256, 256)),
     transforms.ToTensor(),
     transforms.Normalize(norm_mean, norm_std),
])



testset = torchvision.datasets.ImageFolder(root='../../../imagenet2012/',
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

org_img = []
org_labels = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        org_img.append(images)
        org_labels.append(labels)




In [37]:
print(len(testloader))

782


test

In [39]:
org_img = []
org_labels = []
tmp=0
with torch.no_grad():
    for data in testloader:
        if(tmp<=1):
            images, labels = data
            org_img.append(images)
            org_labels.append(labels)
            print(tmp)
            tmp+=1
        else:
            break

0
1


TypeError: 'DataLoader' object is not subscriptable

In [36]:
print(org_img[0][0])

tensor([[[ 0.0569,  0.1426,  0.3652,  ...,  0.9988,  1.3070,  1.1015],
         [-0.5767,  0.7077,  1.3070,  ...,  1.0673,  1.3070,  1.2214],
         [ 0.5707,  0.1768,  1.0159,  ...,  0.8789,  0.8618,  0.7591],
         ...,
         [ 0.0056, -0.1143, -0.0972,  ...,  0.5364, -0.0287, -0.3198],
         [-0.3027, -0.3541, -0.2342,  ...,  0.1768, -0.2171, -0.3369],
         [-0.4911, -0.5424, -0.3369,  ..., -0.1657, -0.3712, -0.4568]],

        [[ 0.3803,  0.3102,  0.3627,  ...,  1.2556,  1.6232,  1.4482],
         [-0.3901,  0.8880,  1.5882,  ...,  1.3256,  1.6408,  1.5707],
         [ 0.8179,  0.5203,  1.7108,  ...,  1.1506,  1.2206,  1.1331],
         ...,
         [ 0.9755,  0.8529,  0.6078,  ...,  1.0630,  0.2052, -0.1450],
         [ 0.6254,  0.4853,  0.3627,  ...,  0.3978, -0.0574, -0.1275],
         [ 0.2577,  0.2402,  0.4503,  ..., -0.0224, -0.1625, -0.2150]],

        [[-0.4624, -0.3230,  0.3916,  ...,  1.4200,  1.6988,  1.5071],
         [-0.6018,  0.0953,  1.3677,  ...,  1

### Test accuracy of the model

In [24]:
# definition of const
# Small test
st = 5
# Midium test
mt = 10
# Large test
lt = 20
# Full test
ft = 100

# Interrupt


In [20]:
def test_acc_new(model, testloader, test_images, n_image, save_test_images = False):
    correct = 0
    total = 0
    p_labels = []
    saved_img = []
    i = 0
    times=0
    
    for data in testloader:
        images, labels = data
        #print(labels)
        print(len(test_images))
        if total < n_image:
            outputs = model(test_images[i])
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted)

            for j in labels:
                # p_labels.append(predicted)
                if save_test_images and total % 4 == 0:
                    saved_img.append(test_images[total])
                    print("total="+total)
                    
                p_labels.append(predicted)
                if predicted[total%64] == j:
                    correct += 1
                total += 1
            i += 1
            times+=1
        else:
            break
    test_accuracy = (100.0 * correct / total)
    print('Accuracy of the network on the', total, "images is: ", test_accuracy, '%')
    if save_test_images == True:
        return test_accuracy, p_labels, saved_img
    else:
        return test_accuracy, p_labels

In [ ]:
test_accuracy, resnet56_labels = test_acc_new(model, testloader, org_img, st, False)

Accuracy of the network on the 1024 images is:  88.28125 %


In [ ]:
acc_label=[]
for i in org_labels:
    for j in i:
        acc_label.append(j)
temp=0
acc=0
for i in resnet56_labels:
    for j in i:
        if acc_label[temp]==j :
            acc+=1

        temp+=1

total_acc=acc/len(acc_label)
print(acc)
print(len(acc_label))
print(total_acc)

0
50000
0.0


In [ ]:
print(org_labels)


In [ ]:
print(resnet56_labels)

[]


## Attack

### Tool functions

In [6]:
def unnormalize(img, mean = np.array(norm_mean), std = np.array(norm_std)):
    '''
    unnormalize the image that has been normalized with mean and std
    '''
    inverse_mean = - mean/std
    inverse_std = 1/std
    img = transforms.Normalize(mean=-mean/std, std=1/std)(img)
    return img

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

def ax_imshow(ax, img, label):
    img = unnormalize(img)     # unnormalize
    img = np.clip(img, 0., 1.)
    ax.set_xticks([])
    ax.set_yticks([])
    img = np.transpose(img, (1,2,0))
    ax.imshow(img)
    ax.set_title(classes[label])

### Implement torchattacks.FGSM

In [ ]:
test_loader_1=torch.utils.data.DataLoader(testset,batch_size=1,shuffle=True)
epsilons = []
init_eps = 0
accuracies=[]
step = 0.001
n_step = 5


In [7]:
epsilons=[0.001]

In [40]:
print(len(org_img))

2


In [26]:
print(st)

5


In [41]:
attack = torchattacks.FGSM(model, 0.01)
tmp=attack(org_img[i], org_labels[i])

In [44]:
fgsm_img_tmp=[]
fgsm_img_tmp.append(tmp)

In [27]:
eps_times=0
fin_image=[]
fgsm_examples = []
for eps in epsilons:
    visual_examples = 5
    fgsm_img = []
    attack = torchattacks.FGSM(model, eps)
    count = 0
    for i in range(6):
        print(i)
        fgsm_img.append(attack(org_img[i], org_labels[i]))
    fgsm_test_accuracy, fgsm_labels , a_images= test_acc_new(model, testloader, fgsm_img, st, True)
    dataiter = iter(testloader)
    images, labels = dataiter.next()
    accuracies.append(fgsm_test_accuracy)

0


RuntimeError: [enforce fail at C:\b\abs_f0dma8qm3d\croot\pytorch_1669187301762\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 50331648 bytes.

### Conclusion

In [ ]:
plt.figure(figsize=(5,5))
plt.plot([0] + epsilons, [test_accuracy] + accuracies, "*-")
plt.yticks(np.arange(0.0, test_accuracy, step=10))
plt.xticks(np.arange(0.0, max(epsilons), step=max(epsilons)/5))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

Save image

In [52]:
def unnormalize(tensor: Tensor, mean: List[float], std: List[float], inplace: bool = False) -> Tensor:
    """Unnormalize a tensor image with mean and standard deviation.

    Args:
        tensor (Tensor): Tensor image of size (C, H, W) or (B, C, H, W) to be normalized.
        mean (sequence): Sequence of means for each channel.
        std (sequence): Sequence of standard deviations for each channel.
        inplace(bool,optional): Bool to make this operation inplace.

    Returns:
        Tensor: Normalized Tensor image.
    """
    if not isinstance(tensor, torch.Tensor):
        raise TypeError('Input tensor should be a torch tensor. Got {}.'.format(type(tensor)))

    if tensor.ndim < 3:
        raise ValueError('Expected tensor to be a tensor image of size (..., C, H, W). Got tensor.size() = '
                         '{}.'.format(tensor.size()))

    if not inplace:
        tensor = tensor.clone()

    dtype = tensor.dtype
    mean = torch.as_tensor(mean, dtype=dtype, device=tensor.device)
    std = torch.as_tensor(std, dtype=dtype, device=tensor.device)
    if (std == 0).any():
        raise ValueError('std evaluated to zero after conversion to {}, leading to division by zero.'.format(dtype))
    if mean.ndim == 1:
        mean = mean.view(-1, 1, 1)
    if std.ndim == 1:
        std = std.view(-1, 1, 1)
    tensor.mul_(std).add_(mean)
    return tensor
    


In [45]:
for i in range(len(fgsm_img)):
    dir_name=os.getcwd()+'\\FGSM_img\\'
    #os.mkdir(dir_name)
    id=0
    for j in range(len(fgsm_img[i])):
        save_image(fgsm_img[i][j],dir_name+'_%i.png'%(id,))
        id+=1

In [49]:
def denorm(data):
    channel_mean = torch.tensor([0.485, 0.456, 0.406])
    channel_std = torch.tensor([0.229, 0.224, 0.225]) 
    MEAN = [-mean/std for mean, std in zip(channel_mean, channel_std)]
    STD = [1/std for std in channel_std]
    denormalizer = transforms.Normalize(mean=MEAN, std=STD)
    image2 = denormalizer(data)
    return image2


In [53]:
for i in range(len(fgsm_img_tmp)):
    dir_name=os.getcwd()+'\\FGSM_img\\'
    os.mkdir(dir_name)
    id=0
    for j in range(len(fgsm_img_tmp[i])):
        save_image(unnormalize(fgsm_img_tmp[i][j],[0.485, 0.456, 0.406],[0.229, 0.224, 0.225]),dir_name+'_%i.png'%(id,))
        id+=1

c:\Users\19401\anaconda3\envs\PRP\lib\site-packages\ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


TypeError: tensor or list of tensors expected, got <class '__main__.UnNormalize'>

In [47]:
print(len(fgsm_img))

0


In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(a_images[0]))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
imshow(torchvision.utils.make_grid(fgsm_examples[6][1]))

In [ ]:
imshow(torchvision.utils.make_grid(orig[1]))